# Import

In [1]:
import os
import sys
import numpy as np
import pandas as pd
from sklearn import preprocessing


# ‘더치페이 요청에 대한 응답률이 높을수록 더치페이 서비스를 더 많이 사용한다.’
# 라는 가설을 통계적으로 검정해주세요.

# Read CSV files

In [2]:
claim_df = pd.read_csv("dutchpay_claim.csv")
claim_df

,claim_id,claim_at,claim_user_id
0,4420721,2020-02-07 15:29:18,5cbd74112c55a0a
1,4420704,2020-02-07 15:26:54,5cbd74112c55a0a
2,4454342,2020-02-10 19:18:31,f077bc4ec8fd0ef
3,4453683,2020-02-10 18:15:11,f077bc4ec8fd0ef
4,4348820,2020-02-01 09:41:30,f077bc4ec8fd0ef
...,...,...,...
159189,4207527,2020-01-18 19:50:31,7376ace386556ee
159190,4099336,2020-01-08 19:11:43,7376ace386556ee
159191,4022961,2020-01-01 15:32:45,7376ace386556ee
159192,4373324,2020-02-03 15:39:17,89d46f024ae9efd


In [3]:
claim_detail_df = pd.read_csv("dutchpay_claim_detail.csv")
claim_detail_df

,claim_detail_id,claim_id,recv_user_id,claim_amount,send_amount,status
0,12918735,4075714,39476d42bd5f268,4,NaN,CLAIM
1,12918734,4075714,a84a2bf8ab324d3,4,4.0,CHECK
2,12829743,4049182,810b763a8f463a9,11750,NaN,CLAIM
3,12829742,4049182,1fc6c9d423a40ec,11750,NaN,CLAIM
4,12829741,4049182,3fc7171d336cf2f,11750,NaN,CLAIM
...,...,...,...,...,...,...
557639,13039768,4110330,b635cf69587bcee,109700,NaN,CLAIM
557640,12023359,3818400,44e8b374daae3c9,43000,43000.0,SEND
557641,12023360,3818400,a30745e0e67dc6c,43000,NaN,CLAIM
557642,12023358,3818400,cf94f7ecacaab18,43000,NaN,CLAIM


In [4]:
users_df = pd.read_csv("users.csv")
users_df

,user_id,gender_cd,age,foreigner_yn,os_type
0,01509a0865440e0,2,32,N,A
1,7d4697fbadb1c09,1,27,N,A
2,7b2a7724faf1400,1,26,N,A
3,b3876137175bec2,1,25,N,B
4,4e1d1005ebb3119,2,31,N,A
...,...,...,...,...,...
123919,9c778efcbbe8f56,1,27,N,A
123920,89d46f024ae9efd,1,24,Y,B
123921,e8ddfdc0ed4096a,2,27,N,A
123922,1619108395b45bd,1,32,N,B


In [5]:
a_payment_trx_df = pd.read_csv("a_payment_trx.csv")
a_payment_trx_df

,id,transaction_id,transacted_at,payment_action_type,user_id,amount
0,50df1872dc3b11e9a1e3509a4c8d0e6f19080,3120268,2019-12-21 15:44:53,PAYMENT,3c9c23defcaedf5,900
1,dfe058efdc7f11e9b6a3509a4c8d0cf318080,3144456,2019-12-21 23:55:38,PAYMENT,3c9c23defcaedf5,900
2,cf51358ee5c311e9b253509a4c8d0e6f18080,3863601,2020-01-03 18:54:37,PAYMENT,3c9c23defcaedf5,3600
3,8f372f4ee59f11e9b253509a4c8d0e6f18080,3849471,2020-01-03 14:35:08,PAYMENT,3c9c23defcaedf5,3600
4,95f7716d088711eab874509a4c8d103b19080,6740655,2020-02-17 00:41:42,PAYMENT,a457f210e06c56a,5900
...,...,...,...,...,...,...
30725,c9c4be0b10cf11eabbe7509a4c8d0e6f18080,7495190,2020-02-27 13:38:42,PAYMENT,705efb8ba0fc4ba,69000
30726,a959181110d811eab28e509a4c8d0caf19080,7498688,2020-02-27 14:42:13,PAYMENT,705efb8ba0fc4ba,69000
30727,bfffa4b210d211ea8031509a4c8d103b18080,7496379,2020-02-27 13:59:54,PAYMENT,705efb8ba0fc4ba,69000
30728,0ce4b38010d311ea87b5509a4c8d0cf318080,7496491,2020-02-27 14:02:03,PAYMENT,705efb8ba0fc4ba,48000


# Review claim_detail status

In [6]:
for i, df in enumerate(claim_detail_df.groupby("claim_id")):
    print(df[1])
    
    if i > 10:
        break

        claim_detail_id  claim_id     recv_user_id  claim_amount  send_amount  \
340939         11640202   3710950  3f2d1ead5ffb3ac          8666       8666.0   
340940         11640204   3710950  182a394e78f03f1          8666          NaN   
340941         11640203   3710950  a9d0143610380c2          8666          NaN   

       status  
340939  CHECK  
340940  CLAIM  
340941  CLAIM  
        claim_detail_id  claim_id     recv_user_id  claim_amount  send_amount  \
132853         11640236   3710962  edb50a3b07d9fcb          7000       7000.0   
132854         11640238   3710962  8ce9978fcb2b71d          7000       7000.0   
132855         11640237   3710962  7301760c242aa4b          7000       7000.0   

       status  
132853  CHECK  
132854   SEND  
132855   SEND  
        claim_detail_id  claim_id     recv_user_id  claim_amount  send_amount  \
527274         11640239   3710963  804774cd0205ee6          1530       1530.0   

       status  
527274  CHECK  
        claim_detail_id  cl

### 자기자신 케이스

In [7]:
claim_id = 3710963

In [8]:
claim_df[claim_df["claim_id"] == claim_id]

,claim_id,claim_at,claim_user_id
75734,3710963,2019-12-01 00:02:23,804774cd0205ee6


In [9]:
claim_detail_df[claim_detail_df["claim_id"] == claim_id]

,claim_detail_id,claim_id,recv_user_id,claim_amount,send_amount,status
527274,11640239,3710963,804774cd0205ee6,1530,1530.0,CHECK


### 1명 송금요청 케이스

In [10]:
claim_id = 3711010

In [11]:
claim_df[claim_df["claim_id"] == claim_id]

,claim_id,claim_at,claim_user_id
52873,3711010,2019-12-01 00:07:36,9b2956f3779fffc


In [12]:
claim_detail_df[claim_detail_df["claim_id"] == claim_id]

,claim_detail_id,claim_id,recv_user_id,claim_amount,send_amount,status
421266,11640393,3711010,b27648cb9614779,10000,10000.0,SEND


### 2명 이상 더치페이 케이스

In [13]:
claim_id = 3711012

In [14]:
claim_df[claim_df["claim_id"] == claim_id]

,claim_id,claim_at,claim_user_id
28190,3711012,2019-12-01 00:07:44,fc475584f568eb5


In [15]:
claim_detail_df[claim_detail_df["claim_id"] == claim_id]

,claim_detail_id,claim_id,recv_user_id,claim_amount,send_amount,status
224646,11640399,3711012,49e35c4ede65249,5260,5260.0,SEND
224647,11640400,3711012,a0e79c13271e687,5260,NaN,CLAIM
224648,11640398,3711012,fc475584f568eb5,5260,5260.0,CHECK
224649,11640402,3711012,20f3cb0f3308a94,5260,NaN,CLAIM
224650,11640401,3711012,3e5492c836be07a,5260,NaN,CLAIM


### CHECK status review

In [16]:
claim_detail_df[claim_detail_df["status"] == "CHECK"]["send_amount"].isna().sum()
# status가 CHECK면 send_amount NaN은 없다.

0

In [17]:
(claim_detail_df[claim_detail_df["status"] == "CHECK"]["send_amount"] == 0).sum()
# status가 CHECK일때 send_amount 0 -> 4,646 케이스

4656

In [18]:
claim_detail_df[claim_detail_df["send_amount"] == 0]
# send_amount 0 인 케이스는 4,646건으로 모두 CHECK

,claim_detail_id,claim_id,recv_user_id,claim_amount,send_amount,status
45,15042406,4684361,d3044177ef09929,0,0.0,CHECK
66,15040252,4683754,0750896e3e89f76,0,0.0,CHECK
155,14675549,4580603,5ebd34fa08e4c46,0,0.0,CHECK
261,12467096,3947003,443f70a93eb5d86,0,0.0,CHECK
270,12401625,3927943,4ff996fc268f3da,0,0.0,CHECK
...,...,...,...,...,...,...
557196,13031447,4107848,b06255b6cf7c053,0,0.0,CHECK
557302,13147027,4141696,6c6e98c31d002b6,0,0.0,CHECK
557469,12960903,4087747,d6b58c7c16cfa71,0,0.0,CHECK
557566,14034185,4397589,33a2190a3206321,0,0.0,CHECK


SEND -> 요청받은 돈을 송금함  
CHECK -> 요청한 사람  
CLAIM -> 요청받은 돈을 주지 않음  

# 더치페이 요청에 대한 응답률

In [19]:
# 더치페이 요청자 제외
claim_response_df = claim_detail_df[claim_detail_df["status"] != "CHECK"]

# 결제요청수신자, status 로 groupby count
claim_response_df = claim_response_df.groupby(["recv_user_id", "status"]).count()["claim_detail_id"].to_frame()

# level 1 transpose to column
claim_response_df = claim_response_df.stack().unstack(level=1)

# drop redundant index
claim_response_df.index = claim_response_df.index.droplevel(1)

# remove columns name
claim_response_df.columns.name = ''
claim_response_df.index.name = "user_id"

# fill NaN to 0
claim_response_df = claim_response_df.fillna(0)

# total count
claim_response_df["TOTAL"] = claim_response_df["CLAIM"] + claim_response_df["SEND"]

# 응답률
claim_response_df["SEND_RATE"] = (claim_response_df["SEND"] / claim_response_df["TOTAL"]).round(3)

# type casting
dtype_dic = {"CLAIM": "int64", "SEND": "int64", "TOTAL": "int64"}
claim_response_df = claim_response_df.astype(dtype_dic)

# drop claim column
claim_response_df = claim_response_df.drop(columns=["CLAIM", "SEND", "TOTAL"])

claim_response_df

,SEND_RATE
user_id,
000109bfc821bda,0.000
0001220f2933b14,0.000
00017188c9f4198,0.000
0001917bc1a49b5,0.000
0001d33a536909b,0.333
...,...
fffef842c35250b,0.000
ffff6651e0ad7de,0.500
ffff6b7cec134f4,0.000


# 더치페이 사용정보

In [20]:
# dutchpay user claim df
claim_df = pd.read_csv("dutchpay_claim.csv")
claim_df = claim_df.groupby(["claim_user_id"]).count()["claim_id"].rename("CLAIM").to_frame()
claim_df.index.name = "user_id"

# total user claim df
users_user_series = pd.read_csv("users.csv")["user_id"]
clain_detail_user_series = pd.read_csv("dutchpay_claim_detail.csv")["recv_user_id"]
total_user_series = users_user_series.append(clain_detail_user_series).rename("user_id")
total_user_df = total_user_series.to_frame().drop_duplicates()
total_user_df["CLAIM"] = 0
total_user_df = total_user_df.set_index("user_id")
total_user_df

# merge counts
totla_claim_df = total_user_df + claim_df
totla_claim_df = totla_claim_df.fillna(0)
totla_claim_df

,CLAIM
user_id,
000109bfc821bda,0.0
0001220f2933b14,0.0
0001440cbac4d21,1.0
00017188c9f4198,0.0
0001917bc1a49b5,0.0
...,...
ffff6b7cec134f4,0.0
ffff6fc24f28db1,0.0
ffffc4e9501da7c,3.0


# 통계 검정  
 <br>
"더치페이 요청에 대한 응답률 정도" 은 "CHECK status는 제외하고 SEND 의 rate 순위" 로 측정 -> 비모수<br>
"더치페이 서비스 이용 정도" 는 "정해진 시간단위의 더치페이 요청 횟수 순위"로 측정 -> 비모수<br>
<br>
비모수 상관 검정 -> Spearman correlation, Kendall correlation<br>
<br>
귀무가설 H0 : 응답률 정도와 서비스 이용 정도는 상관관계가 없다<br>
대립가설 H1 : 응답률 정도와 서비스 이용 정도는 상관관계가 있다<br>
<br>

In [21]:
from scipy.stats import kendalltau
from scipy.stats import spearmanr

In [22]:
# inner join
test_df = pd.merge(totla_claim_df, claim_response_df, left_index=True, right_index=True)
test_df

,CLAIM,SEND_RATE
user_id,,
000109bfc821bda,0.0,0.000
0001220f2933b14,0.0,0.000
00017188c9f4198,0.0,0.000
0001917bc1a49b5,0.0,0.000
0001d33a536909b,0.0,0.333
...,...,...
fffef842c35250b,0.0,0.000
ffff6651e0ad7de,0.0,0.500
ffff6b7cec134f4,0.0,0.000


In [23]:
# test_df["CLAIM"].plot.hist()
# test_df["SEND_RATE"].plot.hist()

In [24]:
kendalltau(test_df["CLAIM"], test_df["SEND_RATE"])

KendalltauResult(correlation=0.03384599106566753, pvalue=2.0461386062294375e-56)

In [25]:
spearmanr(test_df["CLAIM"], test_df["SEND_RATE"])

SpearmanrResult(correlation=0.035229187606778394, pvalue=1.7223842776315469e-56)

*p-value 가 낮아서 유의하긴 하지만 correlation이 0에 가까워서 명확한 상관관계가 보이지 않는다.*